You will work with data from the [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course). 

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

### Step 1:
- Read the test and train datasets
- Separate the `SalePrice` as target variable
- Create a train and validation dataset from the train dataset that you created in the first step
- Check if you have any categorical features
- Check if you have any numerical features

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
# Read in the data

X = pd.read_csv("train.csv", index_col='Id')
X_test = pd.read_csv("test.csv", index_col='Id')

# Remove rows with missing target vaiable

X.dropna(axis=0, subset=['SalePrice'], inplace=True)

# Separate target variable from features

y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

In [4]:
# Split off validation set from training data

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state=0)


In [5]:
# Select categorical columns with low cardinality (few unique values)

categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and
                   X_train[cname].dtype == "object"]

# Select numerical columns

numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

# Keep only selected columns

cols = categorical_cols + numerical_cols
X_train = X_train[cols].copy()
X_valid = X_valid[cols].copy()
X_test = X_test[cols].copy()

In [6]:
X_train.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,774,0,108,0,0,260,0,0,7,2007
871,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,PosN,Norm,...,308,0,0,0,0,0,0,0,8,2009
93,RL,Pave,Grvl,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,...,432,0,0,44,0,0,0,0,8,2009
818,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,...,857,150,59,0,0,0,0,0,7,2008
303,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,843,468,81,0,0,0,0,0,1,2006


### Step 2:
- Create a pipeline of `SimpleImputer` and `StandardScaler` transformers for the numerical data
- Create a `SimpleImputer` and `OneHotEncoder` for the categorical data
- Bundle the pre-processing steps into a column transformer
- Create a RandomForestRegressor

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [8]:
# Preprocessing numerical data

numerical_transf = Pipeline(steps = [('imputer', SimpleImputer(strategy='median')),
                                    ('scaler', StandardScaler())])

# Preprocessing for categorical data

categorical_transf = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
])

# Combine preprocessing for numerical and categorical data

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transf, numerical_cols),
        ('cat', categorical_transf, categorical_cols)
    ])

# Define Model

model = RandomForestRegressor(n_estimators=100, random_state=0)

# Combine preprocessing and modeling in a pipeline

clf = Pipeline(steps = [('preprocessor', preprocessor),
                       ('model', model)])

# Preprocess training data, fit model

clf.fit(X_train, y_train)

# Preprocess validation data, get predictions

preds = clf.predict(X_valid)

print("MAE: ", mean_absolute_error(y_valid, preds))

MAE:  17537.385034246574
